In [ ]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
import keras.utils as image
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import Adam

In [ ]:
# All Categories from dataset
category = ["Basil", "Bok choy", "HolyBasil", "Kale"]
labels_dict_train = dict(category = [], filepath = [])

In [ ]:
def insert_directory (category_type):
    dir_address = '/kaggle/input/vegetable-fulldataset/VegetableDataset/' + str(category_type)
    for dirname, _, filenames in os.walk(dir_address):
        for filename in filenames:
            labels_dict_train['category'].append(category_type)
            labels_dict_train['filepath'].append(os.path.join(dirname, filename))

In [ ]:
for i in range (0,4):
    insert_directory(category[i])

In [ ]:
df_train = pd.DataFrame(labels_dict_train)

In [ ]:
df_train.head()

In [ ]:
print(df_train)

In [ ]:
X_data = np.zeros((len(df_train), 224, 224, 3), dtype='float32')
# One hot encoding
Y_data = label_binarize(df_train['category'], classes = category)

# Reading and converting image to numpy array and normalizing dataset
for i in tqdm(range(len(df_train))):
    img = image.load_img(df_train['filepath'][i], target_size=(224, 224))
    img = image.img_to_array(img)
    x = np.expand_dims(img.copy(), axis=0)
    X_data[i] = x / 255.0

# Printing train image and one hot encode shape & size
print('\nTrain Images shape: ',X_data.shape,' size: {:,}'.format(X_data.size))
print('One-hot encoded output shape: ',Y_data.shape,' size: {:,}'.format(Y_data.size))

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 64, kernel_size = (5,5), activation ='relu', input_shape = (224,224,3)))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 32, kernel_size = (3,3), activation ='relu', kernel_regularizer = 'l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 16, kernel_size = (7,7), activation ='relu', kernel_regularizer = 'l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 8, kernel_size = (5,5), activation ='relu', kernel_regularizer = 'l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(256, activation = "relu", kernel_regularizer = 'l2'))
model.add(Dense(128, activation = "relu", kernel_regularizer = 'l2'))
model.add(Dense(len(category), activation = "softmax"))

model.compile(loss = 'categorical_crossentropy', optimizer = Adam(0.0001),metrics=['accuracy'])

model.summary()


In [ ]:
X_train_and_val, X_test, Y_train_and_val, Y_test = train_test_split(X_data, Y_data, test_size = 0.1)
X_train, X_val, Y_train, Y_val = train_test_split(X_train_and_val, Y_train_and_val, test_size = 0.2)


In [ ]:
epochs = 200
batch_size = 64

history = model.fit(X_train, Y_train, batch_size = batch_size, epochs = epochs,
                    validation_data = (X_val, Y_val))

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(history.history['accuracy'], color='r')
plt.plot(history.history['val_accuracy'], color='b')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'val'])

plt.show()